https://shashacode.tistory.com/63

https://pleasestudy-alswldi.tistory.com/122

In [ ]:
!pip install transformers
!pip install sentencepiece

In [ ]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 7.7 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292695 sha256=402ef1d7fb4826ec1fad9ef686608ef633c21a222de86694edf616c4e872ef60
  Stored in directory: /root/.cache/pip/wheels/d4/33/10/735728ff9ab4891a97789d842c70354bdb368715e35f18fb9e
Successfully built gluonnlp


In [ ]:
!pip uninstall numpy -y
!pip install numpy==1.23.1

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 32.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for numpy: filename=numpy-1.23.1-cp311-cp311-linux_x86_64.whl size=19731415 sha256=1f838a5ce119e759c97f1e27e3e0a188f4068a262afcc4fc75cff9538588507c
  Stored in directory: /root/.cache/pip/wheels/54/67/ec/c3e57b4b51328fb39dd4d63906b0d3bd37a312508e5922682f
Successfully built numpy
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.4.33 requires numpy>=1.24, but you have numpy 1.23.1 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.1 which is incompatible.
plotnine 0.14.5 req

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-soud47t2/kobert-tokenizer_27480aaee6e5470597967d767af38afb
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-soud47t2/kobert-tokenizer_27480aaee6e5470597967d767af38afb
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 5c46b1c68e4755b54879431bd302db621f4d2f47
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=898b82fde9051c0342f11e236a7725218c2921fa6b6492cb420f55ae55c19328
  Stored in directory: /tmp/pip-ephem-wheel-cache-ohjbjprf/wheels/b7/95/7c/3f38b039ae9422cc4a51702b37e352d0585442cbe60285f1f2
Successfully built kobert_tokenizer


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import json

import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import gluonnlp as nlp
from tqdm.notebook import tqdm

from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

/usr/local/lib/python3.11/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [ ]:
device = torch.device("cuda:0")

In [ ]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_path = '/content/drive/MyDrive/Colab Notebooks/data/속성기반_감정분석_데이터'

In [ ]:
train_df = pd.read_csv(data_path + '/train.csv')
test_df = pd.read_csv(data_path + '/test.csv')

In [ ]:
train_df = train_df[['RawText', 'GeneralPolarity']]
test_df = test_df[['RawText', 'GeneralPolarity']]

In [ ]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

<ipython-input-11-2dd1809705e6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.dropna(inplace=True)


In [ ]:
# GeneralPolarity 값 매핑
train_df['GeneralPolarity'] = train_df['GeneralPolarity'].map({1: 0, -1: 1, 0: 2})
test_df['GeneralPolarity'] = test_df['GeneralPolarity'].map({1: 0, -1: 1, 0: 2})

<ipython-input-12-1994ce56aa54>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['GeneralPolarity'] = train_df['GeneralPolarity'].map({1: 0, -1: 1, 0: 2})


In [ ]:
train_df['GeneralPolarity'].unique()

array([0, 1, 2])

In [ ]:
dataset_train = []
for q, label in zip(train_df['RawText'], train_df['GeneralPolarity'])  :
    data = []
    data.append(q)
    data.append(str(label))

    dataset_train.append(data)

In [ ]:
dataset_test = []
for q, label in zip(test_df['RawText'], test_df['GeneralPolarity'])  :
    data = []
    data.append(q)
    data.append(str(label))

    dataset_test.append(data)

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [ ]:
class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)

In [ ]:
test_dataset = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)

In [ ]:
test_dataset[0]

(array([   2, 3942, 2235, 6060, 6553, 2186, 6113, 5561, 4204, 6701,   54,
        2298, 7589, 5058, 5859, 3271, 5032, 7086,  517, 6357, 7330, 2912,
        7828,  905,  830, 6701,   54, 2097, 7147, 7389, 2781, 6416, 7101,
        1535, 5330, 6896, 2235, 6006, 1535, 2959, 6136, 5859, 3278, 6999,
          54, 3494, 6036, 3930, 6797, 6553, 3093, 6657, 7330, 4559, 7848,
        6999,   54,    3,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(58, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 0)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=8,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from sklearn.metrics import f1_score

# 정확도 및 F1 Score 측정 함수
def calc_metrics(X, Y):
    # 예측 결과와 실제 레이블 비교
    max_vals, max_indices = torch.max(X, 1)
    preds = max_indices.cpu().numpy()
    labels = Y.cpu().numpy()

    # 정확도 계산
    accuracy = (preds == labels).sum() / preds.shape[0]

    # F1 Score 계산 (다중 클래스 지원)
    f1 = f1_score(labels, preds, average='macro')  # 'macro'는 클래스별 평균

    return accuracy, f1

In [ ]:
from tqdm import tqdm, tqdm_notebook

train_history = []
test_history = []
loss_history = []

for e in range(num_epochs):
    train_acc = 0.0
    train_f1 = 0.0
    test_acc = 0.0
    test_f1 = 0.0
    model.train()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        # Calculate accuracy and F1 Score for training
        acc, f1 = calc_metrics(out, label)
        train_acc += acc
        train_f1 += f1

        if batch_id % log_interval == 0:
            print(f"epoch {e+1} batch {batch_id+1} loss {loss.data.cpu().numpy()} train acc {train_acc / (batch_id+1)} train f1 {train_f1 / (batch_id+1)}")
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())

    print(f"epoch {e+1} train acc {train_acc / (batch_id+1)} train f1 {train_f1 / (batch_id+1)}")

    # Evaluation phase
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        # Calculate accuracy and F1 Score for testing
        acc, f1 = calc_metrics(out, label)
        test_acc += acc
        test_f1 += f1

    print(f"epoch {e+1} test acc {test_acc / (batch_id+1)} test f1 {test_f1 / (batch_id+1)}")
    test_history.append(test_acc / (batch_id+1))

<ipython-input-27-bcfb6b85dd4b>:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/2867 [00:00<?, ?it/s]

epoch 1 batch 1 loss 2.480375289916992 train acc 0.078125 train f1 0.021645021645021644
epoch 1 batch 201 loss 0.4982861578464508 train acc 0.574238184079602 train f1 0.23227216754064203
epoch 1 batch 401 loss 0.41352587938308716 train acc 0.6875779301745636 train f1 0.32263950058088364
epoch 1 batch 601 loss 0.3577888607978821 train acc 0.7250935940099834 train f1 0.4504513238807401
epoch 1 batch 801 loss 0.2337271124124527 train acc 0.7517556179775281 train f1 0.5088275750265532
epoch 1 batch 1001 loss 0.07833843678236008 train acc 0.7754901348651349 train f1 0.5447703138470714
epoch 1 batch 1201 loss 0.4303897023200989 train acc 0.7881583055786844 train f1 0.5762546718124387
epoch 1 batch 1401 loss 0.309163361787796 train acc 0.7990051748750893 train f1 0.600947338625088
epoch 1 batch 1601 loss 0.29554980993270874 train acc 0.8100015615240475 train f1 0.6177943957240396
epoch 1 batch 1801 loss 0.7738484740257263 train acc 0.8203168378678511 train f1 0.6446244772680131
epoch 1 batch 

<ipython-input-27-bcfb6b85dd4b>:42: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/389 [00:00<?, ?it/s]

...

epoch 3 batch 1 loss 0.16849572956562042 train acc 0.96875 train f1 0.6134920634920634
epoch 3 batch 201 loss 0.3140907883644104 train acc 0.7234141791044776 train f1 0.4659682206748122
epoch 3 batch 401 loss 0.1368436962366104 train acc 0.8148379052369077 train f1 0.5806981509422012
epoch 3 batch 601 loss 0.1916288286447525 train acc 0.8379783693843594 train f1 0.6666993914368577
epoch 3 batch 801 loss 0.086153544485569 train acc 0.8561563670411985 train f1 0.7103102782932311
epoch 3 batch 1001 loss 0.038869526237249374 train acc 0.8717844655344655 train f1 0.7374386604612254
epoch 3 batch 1201 loss 0.21296703815460205 train acc 0.8799568068276437 train f1 0.7581013274139956
epoch 3 batch 1401 loss 0.24888160824775696 train acc 0.8865207887223412 train f1 0.7729429757789303
epoch 3 batch 1601 loss 0.1720041036605835 train acc 0.8932503123048094 train f1 0.7854623411041415
epoch 3 batch 1801 loss 0.9782013893127441 train acc 0.8986587312604108 train f1 0.800483394885221
epoch 3 batch 2

<ipython-input-27-bcfb6b85dd4b>:42: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/389 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


epoch 3 test acc 0.8717421229069687 test f1 0.7946564431556015


In [ ]:
torch.save(model, data_path + '/KoBERT_fine_tuning_model.pth')